In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 49.1 MB 148 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595720 sha256=09cc1181aef505b2c89510a5ab4bd306367cfe7a66575ea376eb97bc6f730d18
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.2 MB/s 
     |████████████████████████████████| 769 kB 4.9 MB/s 
     |████████████████████████████████| 880 kB 54.4 MB/s 
     |████████████████████████████████| 3.0 MB 53.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=eae017515a7fe56902673b65e47958bf47195b2dc2db0e5bd133274182b9d98d
  Stored in directory: /root/.

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-0bazapc7
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-0bazapc7
     |████████████████████████████████| 132 kB 5.2 MB/s 
     |████████████████████████████████| 4.5 MB 23.6 MB/s 
     |████████████████████████████████| 4.2 MB 57.0 MB/s 
     |████████████████████████████████| 6.6 MB 30.8 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 62.4 MB/s 
     |████████████████████████████████| 8.8 MB 52.8 MB/s 
     |████████████████████████████████| 79 kB 10.8 MB/s 
     |████████████████████████████████| 127 kB 74.0 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=67a5dae106c5fc60444975635df9c016651e1ac3281cace649811d5a16bb9810
  Stored in directory: /tmp/pip-ephem-wheel-cache-6rnq5eiz/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd0

In [10]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

In [4]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers == HuggingFace에서 제공하는 transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
#GPU 사용
device = torch.device("cuda:0")

In [6]:
#KoBERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 데이터 로드

In [34]:
from google.colab import files
file_uploaded = files.upload()

Saving text_data.xlsx to text_data.xlsx


In [35]:
data = pd.read_excel('text_data.xlsx')

In [45]:
df = data.iloc[:,:2]

In [47]:
df['Emotion'].value_counts()

행복    6037
슬픔    5267
중립    4830
Name: Emotion, dtype: int64

# 전처리

In [48]:
df.loc[(df['Emotion']=='중립'),'Emotion'] = 0
df.loc[(df['Emotion']=='행복'),'Emotion'] = 1
df.loc[(df['Emotion']=='슬픔'),'Emotion'] = 2

In [49]:
df['Emotion'].value_counts()

1    6037
2    5267
0    4830
Name: Emotion, dtype: int64

In [50]:
df.to_csv('encoded_df.csv', encoding='utf-8')

In [51]:
##nlp.data.TSVDataset()

data_list = []
for q, label in zip(df['Sentence'], df['Emotion']):
    dataset = []
    dataset.append(q)
    dataset.append(str(label))

    data_list.append(dataset)

In [52]:
data_list[:10]

[['어깨라도넓엇으면 옷빨이라도 잘살텐데..', '2'],
 [' 그저 보고싶어죽겟어요', '2'],
 ['아 진짜 저한테는 악운만 붙는거같네요ㅠㅠ........0', '2'],
 [' 지금 계단에서울고있어요', '2'],
 ['저때문에 택배하나 배달하려다 변을 당하신건 아니겠죠0', '2'],
 ['잊은줄 알았는데 잊을수 있다고 생각 했는데갑자기 쏟아지는눈물에  강남한복판에서 주저앉게 되더라..', '2'],
 ['거의 처음 연애였고 처음엔 그 사람이 시작했지만 마지막엔 저만 질질 끌려다니는 연애였네요.', '2'],
 ['근데 같이 갈 사람이 없다는게 안타깝네요..ㅜㅜ', '2'],
 ['어떻게 해야 제 성별을 안착각하게 할수있을까요?', '2'],
 ['유일하게 남은게 그분 내의나 냉장고에 붙혀진 시 한 편이 고작입니다.', '2']]

In [60]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
train_dataset, test_dataset = train_test_split(data_list, test_size=0.25, random_state=0)

In [61]:
print(len(train_dataset))
print(len(test_dataset))

12100
4034


In [58]:
#KoBERT 모델의 입력가능 형태로 By 토큰화, 정수 인코딩, 패딩

#예시코드 사용 - 사용모듈: BERTSentenceTransform
  #BERTSataset: 토큰화와 패딩을 거친 inputs&labels&attention masks까지 포함하고 있는 TensorDataset을 만들어 줌
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [59]:
# Setting parameters (SKTBrain 예시)
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [64]:
# 토큰화 
tokenizer = get_tokenizer() #kobert.utils
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

#BERTDataset 클래스 이용 > TensorDataset으로 만들어주기
train_data = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)
test_data = BERTDataset(test_dataset, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [66]:
# 만들어진 TensorDataset 확인
train_data[0]
# padding sequence
# len & dtype
# attention mask sequence(중요)
  #BERT에 데이터가 입력되었을 때 attention 함수가 적용되어 연산되는데, 1로 패딩된 값은 연산불필요함을 알려주는 데아터

(array([   2, 3923, 2658, 7850, 1562,  517, 6324, 5439, 6751, 6857,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(11, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [67]:
#DataLoader
  #TensorDataset
  #batch_size=현재 쓰고 있는 GPU의 VRAM에 맞도록 설정(일단 넣어보고 VRAM 부족 메세지 뜨면 8배수 중 더 작은 것으로 줄여나가기)
  #num_workers = 학습 도중 CPU 작업을 몇개의 코어를 사용해서 진행할 것인지 
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 모델링

In [68]:
# num_classes: 3

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [69]:
#모델 불러오기
#GPU 사용할 경우 .to(device) 필요
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) 

In [70]:
#학습을 위한 optimizer와 scheduler 선언
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [71]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 모델 학습

In [72]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/190 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.1397861242294312 train acc 0.265625
epoch 1 train acc 0.6504934210526315


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 validation acc 0.825927734375


  0%|          | 0/190 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.1551738679409027 train acc 0.96875
epoch 5 train acc 0.969078947368421


  0%|          | 0/64 [00:00<?, ?it/s]

epoch 5 validation acc 0.818603515625
